In [1]:
from pyspark.sql import *
from pyspark.sql.types import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1667324972983_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [105]:
#reading trade parquet file for particular date
df = spark.read.parquet("/HdiNotebooks/trade/trade_dt={}".format("2020-08-06"))

In [107]:
df.show(50)

+----------+------+--------+--------------------+------------+--------------------+------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|             file_tm|bid_pr|
+----------+------+--------+--------------------+------------+--------------------+------+
|2020-08-06|  SYMB|    NYSE|2020-08-06 16:57:...|          60|2020-08-06 09:30:...|    33|
|2020-08-06|  SYMB|    NYSE|2020-08-06 18:06:...|          70|2020-08-06 09:30:...|    34|
|2020-08-06|  SYMB|    NYSE|2020-08-06 19:21:...|          80|2020-08-06 09:30:...|    33|
|2020-08-06|  SYMB|    NYSE|2020-08-06 20:36:...|          90|2020-08-06 09:30:...|    33|
|2020-08-06|  SYMB|    NYSE|2020-08-06 21:46:...|         100|2020-08-06 09:30:...|    36|
|2020-08-06|  SYMC|    NYSE|2020-08-06 10:42:...|          10|2020-08-06 09:30:...|   157|
|2020-08-06|  SYMC|    NYSE|2020-08-06 11:52:...|          20|2020-08-06 09:30:...|   160|
|2020-08-06|  SYMC|    NYSE|2020-08-06 13:01:...|          30|2020-08-06 09:30:...|   158|

In [108]:
# Create a temp view for df
df.createOrReplaceTempView("trades")

In [109]:
# Create a new dataframe
temp_df = spark.sql("""
            SELECT trade_dt, symbol,
                    exchange,
                   event_tm, 
                   event_seq_nb,
                   bid_pr AS trade_pr
              FROM trades 
              WHERE trade_dt = '2020-08-06'

            """)

In [110]:
#creating temporary view table
temp_df.createOrReplaceTempView("tmp_trade_moving_avg")

## Creating 30-min moving average table

In [111]:
#calculating 30-min moving average
mov_avg_df = spark.sql("""select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr,
        AVG(trade_pr) OVER (PARTITION BY (symbol)
                ORDER BY CAST(event_tm AS timestamp) 
                RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr    
    FROM tmp_trade_moving_avg
""")

In [113]:
mov_avg_df.write.saveAsTable("tem_trade_moving_avg") #Saving temp view table into hive table

In [114]:
mov_avg_df.show(5)

+----------+------+--------+--------------------+------------+--------+----------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|trade_pr|mov_avg_pr|
+----------+------+--------+--------------------+------------+--------+----------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 10:49:...|          10|      74|   74.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:00:...|          20|      76|   76.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 13:11:...|          30|      77|   77.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 14:27:...|          40|      77|   77.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:39:...|          50|      76|   76.0000|
+----------+------+--------+--------------------+------------+--------+----------+
only showing top 5 rows

## Processing previous date data

In [116]:
prev_date_str = "2020-08-05"
df = spark.sql("select trade_dt,exchange, symbol, event_tm, event_seq_nb, bid_pr AS trade_pr from trades where trade_dt = '{}'".format(prev_date_str))

df.createOrReplaceTempView("tmp_last_trade")

In [117]:
last_prev_df = spark.sql("""
        SELECT a.symbol, a.exchange, a.last_pr FROM (
            SELECT symbol, exchange, event_tm, event_seq_nb, trade_pr,
            AVG(trade_pr) OVER (PARTITION BY (symbol)
            ORDER BY CAST(event_tm AS timestamp) 
            RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as last_pr
             FROM tmp_trade_moving_avg
        ) a
""")

In [119]:
# Save Temp View Into Hive Table For Staging
last_prev_df.write.saveAsTable("tmp_last_trade")



In [120]:
last_prev_df.show(5)

+------+--------+-------+
|symbol|exchange|last_pr|
+------+--------+-------+
|  SYMA|    NYSE|74.0000|
|  SYMA|    NYSE|76.0000|
|  SYMA|    NYSE|77.0000|
|  SYMA|    NYSE|77.0000|
|  SYMA|    NYSE|76.0000|
+------+--------+-------+
only showing top 5 rows

Now that we have produced both staging tables, we join them with the main table “quotes” to
populate trade related information. We need to join “quotes” and “temp_trade_moving_avg” to populate trade_pr and mov_avg_pr into quotes. However, we cannot use equality join in this case; trade events don’t happen at the
same quote time.  A good method for this problem is to merge both tables in a common time sequence

## Populating The Latest Trade and Latest Moving Average Trade Price To The Quote Records

In [88]:
quote = spark.read.parquet("/HdiNotebooks/quote/trade_dt={}".format("2020-08-06"))
quote_common = spark.read.parquet("/HdiNotebooks/output_dir/partition=Q")


In [94]:
quote_common.show()

+----------+--------------------+-----------+------+--------------------+------------+--------+------+--------+------+--------+
|  trade_dt|             file_tm|record_type|symbol|            event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|
+----------+--------------------+-----------+------+--------------------+------------+--------+------+--------+------+--------+
|2020-08-06|2020-08-06 09:30:...|          Q|  SYMA|2020-08-06 09:38:...|           1|  NASDAQ|    78|     100|    80|     100|
|2020-08-06|2020-08-06 09:30:...|          Q|  SYMA|2020-08-06 09:46:...|           2|  NASDAQ|    77|     100|    77|     100|
|2020-08-06|2020-08-06 09:30:...|          Q|  SYMA|2020-08-06 09:52:...|           3|  NASDAQ|    79|     100|    79|     100|
|2020-08-06|2020-08-06 09:30:...|          Q|  SYMA|2020-08-06 09:58:...|           4|  NASDAQ|    76|     100|    77|     100|
|2020-08-06|2020-08-06 09:30:...|          Q|  SYMA|2020-08-06 10:07:...|           5|  NASDAQ|    77|  

In [92]:
quote_common.createOrReplaceTempView("quotes")

In [127]:
quote_union = spark.sql('''
    SELECT trade_dt,
            'Q' as record_type,
            symbol,
            event_tm,
            event_seq_nb,
            exchange ,
            bid_pr ,
            bid_size , 
            ask_pr,
            ask_size,
            null as trade_pr, 
            null as mov_avg_pr 
    FROM quotes
    UNION ALL 
    SELECT  trade_dt, 
            'T' as record_type,
            symbol,
            event_tm,
            event_seq_nb,
            exchange,
            null as bid_pr,
            null as bid_size,
            null as ask_pr,
            null as ask_size,
            trade_pr,
            mov_avg_pr 
    FROM tem_trade_moving_avg
'''
)

In [131]:
quote_union.show(5)

+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+
|  trade_dt|record_type|symbol|            event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|mov_avg_pr|
+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+
|2020-08-06|          Q|  SYMA|2020-08-06 09:38:...|           1|  NASDAQ|    78|     100|    80|     100|    null|      null|
|2020-08-06|          Q|  SYMA|2020-08-06 09:46:...|           2|  NASDAQ|    77|     100|    77|     100|    null|      null|
|2020-08-06|          Q|  SYMA|2020-08-06 09:52:...|           3|  NASDAQ|    79|     100|    79|     100|    null|      null|
|2020-08-06|          Q|  SYMA|2020-08-06 09:58:...|           4|  NASDAQ|    76|     100|    77|     100|    null|      null|
|2020-08-06|          Q|  SYMA|2020-08-06 10:07:...|           5|  NASDAQ|    77|     100|    79|     100|    n

In [132]:
quote_union.createOrReplaceTempView("quote_union")

In [135]:
# Populate The Latest trade_pr and mov_avg_pr
quote_union_update = spark.sql("""
    select *,
    last_value(trade_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_trade_pr,
    last_value(mov_avg_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_mov_avg_pr
    from quote_union
""")

In [136]:
quote_union_update.show()

+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+-------------+---------------+
|  trade_dt|record_type|symbol|            event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|mov_avg_pr|last_trade_pr|last_mov_avg_pr|
+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+-------------+---------------+
|2020-08-05|          Q|  SYMC|2020-08-05 09:35:...|           1|    NYSE|   159|     100|   160|     100|    null|      null|         null|           null|
|2020-08-05|          Q|  SYMC|2020-08-05 09:41:...|           2|    NYSE|   160|     100|   162|     100|    null|      null|         null|           null|
|2020-08-05|          Q|  SYMC|2020-08-05 09:51:...|           3|    NYSE|   160|     100|   161|     100|    null|      null|         null|           null|
|2020-08-05|          Q|  SYMC|2020-08-05 10:00:...|      

In [137]:
quote_union_update.createOrReplaceTempView("quote_union_update")

In [141]:
# Filter For Quote Records

quote_update = spark.sql("""
select trade_dt, symbol, event_tm, event_seq_nb, exchange,
bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr
from quote_union_update
where record_type = 'Q'
""")

In [142]:
quote_update.createOrReplaceTempView("quote_update")

## Join With Table temp_last_trade To Get The Prior Day Close Price

In [144]:
#Join With Table temp_last_trade To Get The Prior Day Close Price
quote_final = spark.sql("""
    SELECT trade_dt, 
           symbol, 
           event_tm, 
           event_seq_nb, 
           exchange,
           bid_pr, 
           bid_size, 
           ask_pr, 
           ask_size, 
           last_trade_pr, 
           last_mov_avg_pr,
           bid_pr - close_pr as bid_pr_mv,
           ask_pr - close_pr as ask_pr_mv
    FROM ( 
            SELECT /* + BROADCAST(t) */
                q.trade_dt, 
                q.symbol, 
                q.event_tm, 
                q.event_seq_nb, 
                q.exchange,
                q.bid_pr, 
                q.bid_size, 
                q.ask_pr, 
                q.ask_size, 
                q.last_trade_pr,
                q.last_mov_avg_pr,
                t.last_pr AS close_pr
            FROM quote_update q
            LEFT JOIN temp_last_trade t 
               ON (q.symbol = t.symbol AND q.exchange = t.exchange))
""")


In [145]:
# Show us the data so we can pick which trade_date to use
quote_final.show()

+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
|  trade_dt|symbol|            event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|last_trade_pr|last_mov_avg_pr|bid_pr_mv|ask_pr_mv|
+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
|2020-08-05|  SYMC|2020-08-05 09:35:...|           1|    NYSE|   159|     100|   160|     100|         null|           null|   0.0000|   1.0000|
|2020-08-05|  SYMC|2020-08-05 09:35:...|           1|    NYSE|   159|     100|   160|     100|         null|           null|   1.0000|   2.0000|
|2020-08-05|  SYMC|2020-08-05 09:35:...|           1|    NYSE|   159|     100|   160|     100|         null|           null|   0.0000|   1.0000|
|2020-08-05|  SYMC|2020-08-05 09:35:...|           1|    NYSE|   159|     100|   160|     100|         null|           null|  -2.0

In [147]:
# Write The Final Dataframe Into Azure Blob Storage At Corresponding Partition
trade_date = "2020-08-06"

quote_final.write.parquet("/HdiNotebooks/quote-trade-analytical/date={}".format(trade_date))